# Exploration and Debugging <br> 
### The objectives of this notebook are: 
* **Load** a sample data file from either the regular season or playoff;
* **Understand** an initial overview of the data structure;
* **Create** an ipywidget to interact with a few values of a particular game.

## Imports

In [12]:
import json
import ipywidgets as widgets
import numpy as np
import pandas as pd


## Sample Data Loading

In [13]:
data_path = '../data/raw/'

## Exploration of the data structure

## Which side do team play when they are at home?

In [14]:
df = pd.read_csv("../ift6758/data/tidy_data.csv")
df.head(10)

,game_id,season,game_time,period,period_time,team,shooter,goalie,is_goal,shot_type,x_coordinate,y_coordinate,is_empty_net,strength,is_playoff,home_team
0,2019020910,20192020,0:15,1,00:15,Ottawa Senators,Tyler Ennis,Anton Khudobin,False,Backhand,81.0,-7.0,NaN,NaN,False,Ottawa Senators
1,2019020910,20192020,0:18,1,00:18,Ottawa Senators,Thomas Chabot,Anton Khudobin,False,Wrist Shot,50.0,38.0,NaN,NaN,False,Ottawa Senators
2,2019020910,20192020,0:34,1,00:34,Ottawa Senators,Dylan DeMelo,Anton Khudobin,False,Wrist Shot,32.0,-32.0,NaN,NaN,False,Ottawa Senators
3,2019020910,20192020,1:11,1,01:11,Ottawa Senators,Anthony Duclair,Anton Khudobin,False,Wrist Shot,59.0,29.0,NaN,NaN,False,Ottawa Senators
4,2019020910,20192020,1:43,1,01:43,Ottawa Senators,Cody Goloubef,Anton Khudobin,False,Wrist Shot,33.0,-20.0,NaN,NaN,False,Ottawa Senators
5,2019020910,20192020,2:24,1,02:24,Ottawa Senators,Scott Sabourin,Anton Khudobin,False,Wrist Shot,65.0,17.0,NaN,NaN,False,Ottawa Senators
6,2019020910,20192020,2:56,1,02:56,Dallas Stars,Jason Robertson,Craig Anderson,False,Slap Shot,-51.0,-33.0,NaN,NaN,False,Ottawa Senators
7,2019020910,20192020,3:03,1,03:03,Dallas Stars,John Klingberg,Craig Anderson,True,Deflected,-39.0,30.0,False,Even,False,Ottawa Senators
8,2019020910,20192020,3:41,1,03:41,Ottawa Senators,Brady Tkachuk,Anton Khudobin,False,Deflected,83.0,7.0,NaN,NaN,False,Ottawa Senators
9,2019020910,20192020,3:58,1,03:58,Ottawa Senators,Jean-Gabriel Pageau,Anton Khudobin,False,Wrist Shot,44.0,-29.0,NaN,NaN,False,Ottawa Senators


## Feature engineering tests

In [15]:
sub_df = df[df['team'] == df['home_team']]
sub_df = sub_df[sub_df['is_goal']==True]
coordinates = sub_df.groupby(['season','team','period'])['x_coordinate'].mean().reset_index()
coordinates['home_offensive_side']=np.sign(coordinates['x_coordinate'])
coordinates = coordinates.drop(['x_coordinate'],axis=1)


In [16]:
df=pd.merge(df,coordinates, on=['season','team','period'],how='left')
df.head(10)

,game_id,season,game_time,period,period_time,team,shooter,goalie,is_goal,shot_type,x_coordinate,y_coordinate,is_empty_net,strength,is_playoff,home_team,home_offensive_side
0,2019020910,20192020,0:15,1,00:15,Ottawa Senators,Tyler Ennis,Anton Khudobin,False,Backhand,81.0,-7.0,NaN,NaN,False,Ottawa Senators,1.0
1,2019020910,20192020,0:18,1,00:18,Ottawa Senators,Thomas Chabot,Anton Khudobin,False,Wrist Shot,50.0,38.0,NaN,NaN,False,Ottawa Senators,1.0
2,2019020910,20192020,0:34,1,00:34,Ottawa Senators,Dylan DeMelo,Anton Khudobin,False,Wrist Shot,32.0,-32.0,NaN,NaN,False,Ottawa Senators,1.0
3,2019020910,20192020,1:11,1,01:11,Ottawa Senators,Anthony Duclair,Anton Khudobin,False,Wrist Shot,59.0,29.0,NaN,NaN,False,Ottawa Senators,1.0
4,2019020910,20192020,1:43,1,01:43,Ottawa Senators,Cody Goloubef,Anton Khudobin,False,Wrist Shot,33.0,-20.0,NaN,NaN,False,Ottawa Senators,1.0
5,2019020910,20192020,2:24,1,02:24,Ottawa Senators,Scott Sabourin,Anton Khudobin,False,Wrist Shot,65.0,17.0,NaN,NaN,False,Ottawa Senators,1.0
6,2019020910,20192020,2:56,1,02:56,Dallas Stars,Jason Robertson,Craig Anderson,False,Slap Shot,-51.0,-33.0,NaN,NaN,False,Ottawa Senators,-1.0
7,2019020910,20192020,3:03,1,03:03,Dallas Stars,John Klingberg,Craig Anderson,True,Deflected,-39.0,30.0,False,Even,False,Ottawa Senators,-1.0
8,2019020910,20192020,3:41,1,03:41,Ottawa Senators,Brady Tkachuk,Anton Khudobin,False,Deflected,83.0,7.0,NaN,NaN,False,Ottawa Senators,1.0
9,2019020910,20192020,3:58,1,03:58,Ottawa Senators,Jean-Gabriel Pageau,Anton Khudobin,False,Wrist Shot,44.0,-29.0,NaN,NaN,False,Ottawa Senators,1.0


In [18]:
def get_goal_distance(x,y,team,home_team,home_offensive_side):
    goal_coord = np.array([89,0])
    if x is None or y is None:
        return None
    if team == home_team:
        home_offensive_side * goal_coord
    else:
        -1 * home_offensive_side * goal_coord
    return np.linalg.norm(np.array([x,y]) - goal_coord)
df['shot_distance']= df.apply(lambda row: get_goal_distance(
                                       row['x_coordinate'],
                                       row['y_coordinate'],
                                       row['team'],
                                       row['home_team'],
                                       row['home_offensive_side']), axis=1)

df.head(10)

,game_id,season,game_time,period,period_time,team,shooter,goalie,is_goal,shot_type,x_coordinate,y_coordinate,is_empty_net,strength,is_playoff,home_team,home_offensive_side,shot_distance
0,2019020910,20192020,7:36,1,07:36,Ottawa Senators,Jean-Gabriel Pageau,Anton Khudobin,True,Snap Shot,75.0,-2.0,False,Power Play,False,Ottawa Senators,1.0,14.142136
1,2019020910,20192020,18:10,1,18:10,Ottawa Senators,Brady Tkachuk,Anton Khudobin,True,Wrist Shot,82.0,3.0,False,Even,False,Ottawa Senators,1.0,7.615773
2,2019020746,20192020,7:15,1,07:15,Ottawa Senators,Brady Tkachuk,David Rittich,True,Wrist Shot,81.0,37.0,False,Even,False,Ottawa Senators,1.0,37.854986
3,2019020988,20192020,7:13,1,07:13,Ottawa Senators,Connor Brown,Thatcher Demko,True,Wrist Shot,59.0,-28.0,False,Even,False,Ottawa Senators,1.0,41.036569
4,2019020988,20192020,7:44,1,07:44,Ottawa Senators,Bobby Ryan,Thatcher Demko,True,Tip-In,85.0,-9.0,False,Even,False,Ottawa Senators,1.0,9.848858
5,2019021038,20192020,13:57,1,13:57,Ottawa Senators,Connor Brown,Semyon Varlamov,True,Wrist Shot,88.0,4.0,False,Even,False,Ottawa Senators,1.0,4.123106
6,2019020255,20192020,9:34,1,09:34,Ottawa Senators,Filip Chlapik,James Reimer,True,Backhand,77.0,1.0,False,Even,False,Ottawa Senators,1.0,12.041595
7,2019020255,20192020,16:47,1,16:47,Ottawa Senators,Vladislav Namestnikov,James Reimer,True,Tip-In,77.0,-1.0,False,Even,False,Ottawa Senators,1.0,12.041595
8,2019020255,20192020,16:51,1,16:51,Ottawa Senators,Jean-Gabriel Pageau,James Reimer,True,Wrist Shot,58.0,8.0,False,Even,False,Ottawa Senators,1.0,32.015621
9,2019020347,20192020,0:57,1,00:57,Ottawa Senators,Thomas Chabot,Henrik Lundqvist,True,Slap Shot,65.0,24.0,False,Even,False,Ottawa Senators,1.0,33.941125


In [ ]:
## Widget Interactions

## Widget Interactions

from IPython.display import display
w = widgets.IntSlider()
display(w)